In [1]:
from MOT import MOT
import numpy as np
import os
from BfTableGenerator import RansacCollector
from BfTableGenerator import TDmapLoader
from tqdm import tqdm
import open3d as op3
# from open3d.web_visualizer import draw
from Utils import *
import pandas as pd
from scipy.sparse import csr_matrix

In [2]:
params = {
    'd':1.2,
    'thred_s':0.3,
    'N':20,
    'delta_thred' : 1e-3,
    'step':0.1,
    'win_size':(5,11),
    'eps': 1.8,
    'min_samples':15,
    'missing_thred':7,
    'ending_frame' : 6000,
    'background_update_frame':2000,
    'save_pcd' : None,
    'save_Azimuth_Laser_info' : False
}

pcap_path = '../RawLidarData/USAPKWY/USApkwy.pcap'
output_file_path = '../RawLidarData/USAPKWY/'
mot = MOT(pcap_path,output_file_path,**params)
mot.initialization()
mot.mot_tracking(A,P,H,Q,R)
mot.save_result()

  0%|          | 3/2000 [00:00<01:31, 21.73it/s]

Initialization...


100%|██████████| 2000/2000 [01:30<00:00, 22.07it/s]


Generating Threshold Map


Tracking 2000 frame:  33%|███▎      | 1998/5998 [02:50<04:46, 13.97it/s]

...

Tracking 4000 frame:  67%|██████▋   | 3998/5998 [05:30<02:16, 14.66it/s]

Generating Threshold Map


 13%|█▎        | 26/205 [00:00<00:00, 248.54it/s]

Generating Traj Files...


100%|██████████| 205/205 [00:00<00:00, 259.13it/s]


In [186]:
col_info = ['ObjectID','FrameIndex']
col_est = ['Object_Length_est','Object_Width_est','Object_Height_est','Coord_x_est','Coord_y_est','Coord_dis_est','Speed_x','Speed_y','Speed_est']
col_mea = ['Object_Length_mea','Object_Width_mea','Object_Height_mea','Coord_x_mea','Coord_y_mea','Coord_dis_mea']

key = 21
missing_thred = 7
temp = np.array(mot.Off_tracking_pool[key].post_seq)
temp = temp.reshape(temp.shape[0],temp.shape[1])[1:-missing_thred] # exclude first data point 
# [0,1,2,3,4,5,6,10,11]
temp_lwhxy = temp[:,[2,3,4,0,1]]
dis_est = np.sqrt(np.sum(temp_lwhxy[:,[3,4]]**2,axis = 1)).reshape(-1,1)
speed_xy = temp[:,[5,6]]*10  #m/s
speed = np.sqrt(np.sum(speed_xy**2,axis = 1)).reshape(-1,1)*3600/1000
est = np.concatenate([temp_lwhxy,dis_est,speed_xy,speed],axis = 1)
temp = mot.Off_tracking_pool[key].mea_seq
emp = []
for i,vec in enumerate(temp):
    if type(vec) == int:
        emp.append(-np.ones(len(col_mea)-1))
    else:
        emp.append(vec.flatten())
emp = np.array(emp)[1:-missing_thred,[2,3,4,0,1]]
dis_mea = np.sqrt(np.sum(emp[:,[3,4]]**2,axis = 1)).reshape(-1,1)
mea = np.concatenate([emp,dis_mea],axis = 1)
timestp = []
for i in range(len(mea)):
    f = i + mot.Off_tracking_pool[key].start_frame + 1
    timestp.append('%06.0f'%f)
timestp = np.array(timestp).reshape(-1,1)
objid = (np.ones(len(mea)) * key).astype(int).reshape(-1,1)
summary = np.concatenate([objid,timestp,mea,est],axis = 1)

pd.DataFrame(summary,columns=col_info+col_mea+col_est)

,ObjectID,FrameIndex,Object_Length_mea,Object_Width_mea,Object_Height_mea,Coord_x_mea,Coord_y_mea,Coord_dis_mea,Object_Length_est,Object_Width_est,Object_Height_est,Coord_x_est,Coord_y_est,Coord_dis_est,Speed_x,Speed_y,Speed_est
0,21,000482,-1.0,-1.0,-1.0,-1.0,-1.0,1.4142135623730951,1.288290434500336,1.361301055969193,0.5939454616930608,26.395344963282746,-39.5853782834675,47.57852887149745,0.0,0.0,0.0
1,21,000483,1.5626233801937168,1.5384394422516507,0.5947390203011222,26.634075061347367,-39.34641649307777,47.513308085422004,1.5572757594199667,1.534986452265638,0.5947235513224076,26.508642197527784,-39.471971091995265,47.54728817760126,0.7544786758747928,0.7552109124887576,3.843042735542606
2,21,000484,1.5637510820184595,3.247547412945842,0.7277574323764355,26.768003733024464,-38.39135464160576,46.80194584703456,1.5832187661387325,2.9308673629097113,0.7021781718382674,26.704160892802573,-38.761174435348416,47.069532105133504,1.6323834191751936,4.635538040901393,17.692411810821966
3,21,000485,1.4295936328138836,3.3252407409667555,0.7291555883888257,26.779383940014352,-38.25213863667997,46.694341354027934,1.4822196941262542,3.460515021265074,0.7399765437172096,26.817132499480735,-38.245420591809236,46.71049980185496,1.6945617546569711,5.869522251925257,21.993269194011297
4,21,000486,1.2900120830816064,1.3425736654494287,0.5902338450931257,26.84221240017894,-39.065027296168864,47.39810886719784,1.3431651642048708,2.249542523888192,0.6550445174169066,26.90043024700356,-38.53435154419455,46.99499331211222,1.5525992822987638,2.765097505079805,11.416217434954183
5,21,000487,1.3337639112626647,1.3970926788700169,0.5975304154186527,26.947191970656682,-38.98640410110683,47.39294103385254,1.3101103105823708,1.677352994515313,0.616893967847961,26.993640776911235,-38.69874207857053,47.183146152589806,1.467547022800444,1.1059152561590946,6.615329690986613
6,21,000488,1.4081299517162087,1.5920973026797895,0.6054791307561462,27.013026788115532,-38.94535264331367,47.396667696869855,1.34580828883587,1.517301367428075,0.6030648642386057,27.069057404414128,-38.799828751477484,47.30920185236109,1.3156589613730385,0.18347930796719397,4.782208291349217
7,21,000489,-1.0,-1.0,-1.0,-1.0,-1.0,1.4142135623730951,1.3211650371456045,1.284830640475861,0.5868982137098084,27.206074446020544,-38.793720445803835,47.38273137747117,1.424681870755318,-0.061313194494225995,5.133602197208735
8,21,000490,1.374483697846987,3.313208383403591,0.7389217836197446,27.172265334437867,-37.90906189267631,46.64149415475299,1.354545235033177,2.7350049791834854,0.6959077739194971,27.222356213730187,-38.15796922132214,46.8730977525764,1.0951837659122434,1.8729555562699591,7.810747362421079
9,21,000491,1.6538483939601143,4.311462056956408,0.763885716906048,27.494092323150852,-37.37997157614964,46.40245023387987,1.5353258446863065,3.8102698090796667,0.7461181271567743,27.42724080332328,-37.624652489927904,46.56036955471165,1.4329049701769132,3.0485297256887014,12.126577499025464


In [182]:
summary.shape

(31, 17)

In [172]:
timestp'%06.0f.pcd'%f

array([[482],
       [483],
       [484],
       [485],
       [486],
       [487],
       [488],
       [489],
       [490],
       [491],
       [492],
       [493],
       [494],
       [495],
       [496],
       [497],
       [498],
       [499],
       [500],
       [501],
       [502],
       [503],
       [504],
       [505],
       [506],
       [507],
       [508],
       [509],
       [510],
       [511],
       [512]])

In [169]:
mea.shape

(31, 6)

In [167]:
len(timestp)

31

In [ ]:
def get_summary_file(post_seq,mea_seq):
    temp = np.array(post_seq)
    temp = temp.reshape(temp.shape[0],temp.shape[1])[:,[0,1,2,3,4,5,6,10,11]]
    df_ = pd.DataFrame(temp,columns= col_names_)
    temp = mea_seq
    emp = []
    for i,vec in enumerate(temp):
        if type(vec) == int:
            emp.append(-np.ones(len(col_names)).astype(np.int8))
        else:
            emp.append(vec.flatten())
    emp = np.array(emp)
    df = pd.DataFrame(emp,columns = col_names)
    summary = pd.concat([df,df_],axis = 1)
    return summary

In [2]:
os.chdir(r'E:\Data\Verteran')

In [3]:
TrajPath = r'.\OutputFile\OutputTrajs'
TrajLis = os.listdir(TrajPath)

FileNotFoundError: [Errno 2] No such file or directory: '.\\OutputFile\\OutputTrajs'

In [ ]:
TPM = np.zeros((400,400))

In [37]:
TPMs = [np.zeros((400*400,400*400)) for i in range(7)]

MemoryError: Unable to allocate 191. GiB for an array with shape (160000, 160000) and data type float64

In [30]:
# rule out the estimated values by KF
TestTraj = np.array(pd.read_csv(os.path.join(TrajPath,TrajLis[0])))[:,[0,1]][:-8]
BinX,BinY = np.arange(-200,201),np.arange(-200,201)
XIndex,YIndex = np.digitize(TestTraj[:,0],BinX),np.digitize(TestTraj[:,1],BinY)
for tpm in TPMs:
    

In [ ]:
np.zeros((200,200,))

In [20]:
TestTraj.iloc[:-8]

,X_Coord_mea,Y_Coord_mea,X_Len_mea,Y_Len_mea,Z_Len_mea,X_Coord_est,Y_Coord_est,X_Len_est,Y_Len_est,Z_Len_est,X_Vel_est,Y_Vel_est,X_Acc_est,Y_Acc_est
0,-11.880474,-24.119835,5.745061,3.441625,1.998897,-11.880474,-24.119835,5.745061,3.441625,1.998897,0.000000,0.000000,0.000000,0.000000
1,-11.368456,-24.197860,5.664705,3.208388,1.987152,-11.786089,-24.134218,5.668514,3.219442,1.987708,0.062645,-0.009546,0.020882,-0.003182
2,-11.024649,-24.375524,5.496608,3.371619,2.002513,-11.413091,-24.245639,5.512613,3.340198,2.000077,0.291764,-0.082358,0.092412,-0.027100
3,-10.769361,-24.353723,4.778183,2.999653,1.992186,-10.886457,-24.349060,4.904718,3.086393,1.995657,0.496676,-0.113938,0.125370,-0.028413
4,-10.471005,-24.214727,4.444730,2.422467,2.000417,-10.421378,-24.305243,4.464018,2.588782,1.999413,0.573218,-0.053293,0.113066,-0.005970
5,-10.226033,-24.236790,3.776464,2.431757,1.967476,-10.069100,-24.281851,3.874649,2.388622,1.979858,0.557386,-0.022253,0.084361,0.002272
6,-9.911183,-24.461892,3.335502,2.340459,1.958870,-9.739566,-24.400137,3.358502,2.261529,1.964841,0.526517,-0.061446,0.061200,-0.006062
7,-9.634861,-24.415833,2.605300,2.102447,1.946846,-9.446976,-24.436091,2.712378,2.083689,1.950491,0.482500,-0.056163,0.041797,-0.003970
8,-9.321026,-24.570280,1.969080,1.984450,2.114373,-9.155233,-24.536879,2.051161,1.944257,2.041065,0.445053,-0.076098,0.028184,-0.006713
9,-8.997133,-24.676813,1.517818,1.815612,1.647589,-8.858574,-24.648977,1.494903,1.795908,1.823872,0.415482,-0.094414,0.018795,-0.008599


In [ ]:
TPM = np.zeros((200,200))

In [ ]:
np.digitize()